In [50]:
import pandas as pd
import numpy as np
from sklearn import *

import matplotlib.pyplot as plt

%matplotlib inline

In [51]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
del df["Id"]
df = df.dropna()
df.head()

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,LotConfig,LandSlope,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalesPrice
0,60,RL,65.0,8450,Pave,None,Reg,Lvl,Inside,Gtl,...,0,None,None,None,0,2,2008,WD,Normal,208500.0
1,20,RL,80.0,9600,Pave,None,Reg,Lvl,FR2,Gtl,...,0,None,None,None,0,5,2007,WD,Normal,181500.0
2,60,RL,68.0,11250,Pave,None,IR1,Lvl,Inside,Gtl,...,0,None,None,None,0,9,2008,WD,Normal,223500.0
3,70,RL,60.0,9550,Pave,None,IR1,Lvl,Corner,Gtl,...,0,None,None,None,0,2,2006,WD,Abnorml,140000.0
4,60,RL,84.0,14260,Pave,None,IR1,Lvl,FR2,Gtl,...,0,None,None,None,0,12,2008,WD,Normal,250000.0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1460 entries, 0 to 1459
Data columns (total 79 columns):
MSSubClass       1460 non-null int64
MSZoning         1460 non-null object
LotFrontage      1460 non-null float64
LotArea          1460 non-null int64
Street           1460 non-null object
Alley            1460 non-null object
LotShape         1460 non-null object
LandContour      1460 non-null object
LotConfig        1460 non-null object
LandSlope        1460 non-null object
Neighborhood     1460 non-null object
Condition1       1460 non-null object
Condition2       1460 non-null object
BldgType         1460 non-null object
HouseStyle       1460 non-null object
OverallQual      1460 non-null int64
OverallCond      1460 non-null int64
YearBuilt        1460 non-null int64
YearRemodAdd     1460 non-null int64
RoofStyle        1460 non-null object
RoofMatl         1460 non-null object
Exterior1st      1460 non-null object
Exterior2nd      1460 non-null object
MasVnrType       1460 no

In [4]:
target = "SalesPrice"

X = df.drop(columns=target)
X_dummy = pd.get_dummies(X, drop_first=True)

y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X_dummy, y , test_size = 0.3, random_state =1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.LinearRegression())
])


pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)

print("r2 training: ", metrics.r2_score(y_train, y_train_pred))
print("r2 test: ", metrics.r2_score(y_test, y_test_pred))

print("rmse train: ", metrics.mean_squared_error(y_train, y_train_pred) ** 0.5)
print("rmse test: ", metrics.mean_squared_error(y_test, y_test_pred)** 0.5)


r2 training:  0.9494518297088844
r2 test:  -3.0715943247509893e+22
rmse train:  0.08606817747710854
rmse test:  76190488351.17424


In [5]:
est = pipe.steps[-1][-1]
thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "coeff":  est.coef_})
thetas["abs_coeff"] = np.abs(thetas["coeff"])
thetas.sort_values("abs_coeff", ascending=False)

,feature,coeff,abs_coeff
15,GrLivArea,1.501695e+11,1.501695e+11
13,2ndFlrSF,-1.233520e+11,1.233520e+11
12,1stFlrSF,-1.112376e+11,1.112376e+11
219,GarageFinish_None,-1.072428e+11,1.072428e+11
134,Exterior2nd_CBlock,8.844505e+10,8.844505e+10
...,...,...,...
249,SaleType_ConLw,-3.542900e-04,3.542900e-04
45,LotShape_Reg,-2.273321e-04,2.273321e-04
158,Foundation_Slab,-1.544952e-04,1.544952e-04
42,Alley_Pave,-2.288818e-05,2.288818e-05


In [6]:
thetas.columns

Index(['feature', 'coeff', 'abs_coeff'], dtype='object')

In [7]:
m = pd.DataFrame({"actual": y_test, "predicted": y_test_pred})
m.sort_values("predicted", ascending = False)

,actual,predicted
1270,12.468437,1.281787e+12
375,11.018629,9.484743e+11
1003,11.827043,1.510345e+09
1011,11.512925,1.066149e+09
120,12.100712,5.829678e+08
...,...,...
968,10.542706,1.100410e+01
916,10.471950,1.089222e+01
1000,11.314475,1.086910e+01
636,11.002100,1.080757e+01


In [8]:
target = "SalesPrice"

X = df.drop(columns=target)
X_dummy = pd.get_dummies(X, drop_first=True)

y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X_dummy, y , test_size = 0.3, random_state =1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso(alpha=0.01))
])


pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)

print("r2 training: ", metrics.r2_score(y_train, y_train_pred))
print("r2 test: ", metrics.r2_score(y_test, y_test_pred))

print("rmse train: ", metrics.mean_squared_error(y_train, y_train_pred) ** 0.5)
print("rmse test: ", metrics.mean_squared_error(y_test, y_test_pred)** 0.5)

est = pipe.steps[-1][-1]
thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "coeff":  est.coef_})
thetas["abs_coeff"] = np.abs(thetas["coeff"])
thetas.sort_values("abs_coeff", ascending=False)



r2 training:  0.8940480673803641
r2 test:  0.8785389987905549
rmse train:  0.12460754170603186
rmse test:  0.15150859029885974


,feature,coeff,abs_coeff
3,OverallQual,0.104135,0.104135
15,GrLivArea,0.095563,0.095563
25,GarageCars,0.055896,0.055896
5,YearBuilt,0.029465,0.029465
39,MSZoning_RM,-0.026432,0.026432
...,...,...,...
109,RoofStyle_Shed,0.000000,0.000000
110,RoofMatl_CompShg,0.000000,0.000000
111,RoofMatl_Membran,0.000000,0.000000
112,RoofMatl_Metal,0.000000,0.000000


In [9]:
target = "SalesPrice"

X = df.drop(columns=target)
X_dummy = pd.get_dummies(X, drop_first=True)

y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X_dummy, y , test_size = 0.3, random_state =1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(alpha=0.01, l1_ratio=0.1))
])


pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)

print("r2 training: ", metrics.r2_score(y_train, y_train_pred))
print("r2 test: ", metrics.r2_score(y_test, y_test_pred))

print("rmse train: ", metrics.mean_squared_error(y_train, y_train_pred) ** 0.5)
print("rmse test: ", metrics.mean_squared_error(y_test, y_test_pred)** 0.5)

est = pipe.steps[-1][-1]
thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "coeff":  est.coef_})
thetas["abs_coeff"] = np.abs(thetas["coeff"])
thetas.sort_values("abs_coeff", ascending=False)



r2 training:  0.9375912824998169
r2 test:  0.8810679185868064
rmse train:  0.09563404287838835
rmse test:  0.14992302580448208


,feature,coeff,abs_coeff
15,GrLivArea,0.090815,0.090815
110,RoofMatl_CompShg,0.083855,0.083855
3,OverallQual,0.070749,0.070749
114,RoofMatl_Tar&Grv,0.056630,0.056630
116,RoofMatl_WdShngl,0.045924,0.045924
...,...,...,...
150,ExterQual_Gd,-0.000000,0.000000
67,Neighborhood_NPkVill,-0.000000,0.000000
147,MasVnrType_None,-0.000000,0.000000
146,MasVnrType_BrkFace,0.000000,0.000000


In [16]:
10 ** np.linspace(-1, 0, 10) + 0.5

array([0.6       , 0.62915497, 0.66681005, 0.71544347, 0.77825594,
       0.85938137, 0.96415888, 1.09948425, 1.27426368, 1.5       ])

In [17]:
param_grid = {
    "est__l1_ratio": np.linspace(0, 0.1, 10),
    "est__alpha": 10 ** np.linspace(-1, 0, 10) + 0.5
}

gsearch = model_selection.GridSearchCV(pipe, param_grid
            , cv = 5, verbose = True, n_jobs= 8 )

gsearch.fit(X_train, y_train)


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    1.7s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    5.9s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('poly',
                                        PolynomialFeatures(degree=1,
                                                           include_bias=False,
                                                           interaction_only=False,
                                                           order='C')),
                                       ('scaler',
                                        StandardScaler(copy=True,
                                                       with_mean=True,
                                                       with_std=True)),
                                       ('est',
                                        ElasticNet(alpha=0.01, copy_X=True,
                                                   fit_intercept=True,
                                                   l1_ratio=0.1, max_iter=1000,
                   

In [18]:
gsearch.best_score_

0.840933149502801

In [19]:
gsearch.best_params_

{'est__alpha': 0.6, 'est__l1_ratio': 0.011111111111111112}

In [20]:
gsearch.best_estimator_

Pipeline(memory=None,
         steps=[('poly',
                 PolynomialFeatures(degree=1, include_bias=False,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('est',
                 ElasticNet(alpha=0.6, copy_X=True, fit_intercept=True,
                            l1_ratio=0.011111111111111112, max_iter=1000,
                            normalize=False, positive=False, precompute=False,
                            random_state=None, selection='cyclic', tol=0.0001,
                            warm_start=False))],
         verbose=False)

In [21]:
gsearch.best_estimator_[-1]

ElasticNet(alpha=0.6, copy_X=True, fit_intercept=True,
           l1_ratio=0.011111111111111112, max_iter=1000, normalize=False,
           positive=False, precompute=False, random_state=None,
           selection='cyclic', tol=0.0001, warm_start=False)

In [26]:
thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "w": gsearch.best_estimator_[-1].coef_}) 
thetas["w_abs"] = np.abs(thetas.w)
thetas = thetas.sort_values("w_abs", ascending = False)
thetas

,feature,w,w_abs
3,OverallQual,0.047084,0.047084
15,GrLivArea,0.034963,0.034963
25,GarageCars,0.025661,0.025661
22,TotRmsAbvGrd,0.025595,0.025595
12,1stFlrSF,0.022393,0.022393
...,...,...,...
137,Exterior2nd_ImStucc,0.000000,0.000000
138,Exterior2nd_MetalSd,-0.000000,0.000000
139,Exterior2nd_Other,0.000000,0.000000
140,Exterior2nd_Plywood,-0.000000,0.000000


In [41]:
param_grid = {
    "est__alpha": np.abs(np.random.random(10) * 0.01 - 0.0077/2)
}

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.Lasso())
])


gsearch = model_selection.GridSearchCV(pipe, param_grid
            , cv = 5, verbose = True, n_jobs= 8 )

gsearch.fit(X_train, y_train)
gsearch.best_params_

thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "w": gsearch.best_estimator_[-1].coef_}) 
thetas["w_abs"] = np.abs(thetas.w)
thetas = thetas.sort_values("w_abs", ascending = False)
thetas

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=8)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  50 out of  50 | elapsed:    0.8s finished


,feature,w,w_abs
15,GrLivArea,0.095771,0.095771
3,OverallQual,0.092458,0.092458
25,GarageCars,0.051211,0.051211
5,YearBuilt,0.034545,0.034545
4,OverallCond,0.028118,0.028118
...,...,...,...
113,RoofMatl_Roll,-0.000000,0.000000
114,RoofMatl_Tar&Grv,0.000000,0.000000
115,RoofMatl_WdShake,-0.000000,0.000000
117,Exterior1st_AsphShn,0.000000,0.000000


In [42]:
thetas[thetas.w_abs>0].shape

(81, 3)

In [43]:
thetas.shape

(258, 3)

In [45]:
thetas.iloc[:10]

,feature,w,w_abs
15,GrLivArea,0.095771,0.095771
3,OverallQual,0.092458,0.092458
25,GarageCars,0.051211,0.051211
5,YearBuilt,0.034545,0.034545
4,OverallCond,0.028118,0.028118
70,Neighborhood_NridgHt,0.025521,0.025521
60,Neighborhood_Crawfor,0.025394,0.025394
39,MSZoning_RM,-0.022880,0.022880
90,Condition2_PosN,-0.020266,0.020266
16,BsmtFullBath,0.020049,0.020049


In [46]:
import_features = list(thetas.iloc[:10]["feature"])
import_features

['GrLivArea',
 'OverallQual',
 'GarageCars',
 'YearBuilt',
 'OverallCond',
 'Neighborhood_NridgHt',
 'Neighborhood_Crawfor',
 'MSZoning_RM',
 'Condition2_PosN',
 'BsmtFullBath']

In [47]:
import_features = ['GrLivArea',
 'OverallQual',
 'GarageCars',
 'YearBuilt',
 'OverallCond',
 'Neighborhood',
 'MSZoning',
 'Condition2',
 'BsmtFullBath']

In [48]:
df[import_features]

,GrLivArea,OverallQual,GarageCars,YearBuilt,OverallCond,Neighborhood,MSZoning,Condition2,BsmtFullBath
0,1710,7,2.0,2003,5,CollgCr,RL,Norm,1.0
1,1262,6,2.0,1976,8,Veenker,RL,Norm,0.0
2,1786,7,2.0,2001,5,CollgCr,RL,Norm,1.0
3,1717,7,3.0,1915,5,Crawfor,RL,Norm,1.0
4,2198,8,3.0,2000,5,NoRidge,RL,Norm,1.0
...,...,...,...,...,...,...,...,...,...
1455,1647,6,2.0,1999,5,Gilbert,RL,Norm,0.0
1456,2073,6,2.0,1978,6,NWAmes,RL,Norm,1.0
1457,2340,7,1.0,1941,9,Crawfor,RL,Norm,0.0
1458,1078,5,1.0,1950,6,NAmes,RL,Norm,1.0


In [49]:
target = "SalesPrice"

X = df[import_features]
X_dummy = pd.get_dummies(X, drop_first=True)

y = np.log(df[target])

X_train, X_test, y_train, y_test = model_selection.train_test_split(
    X_dummy, y , test_size = 0.3, random_state =1)

pipe = pipeline.Pipeline([
    ("poly", preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ("scaler", preprocessing.StandardScaler()),
    ("est", linear_model.ElasticNet(alpha=0.01, l1_ratio=0.1))
])


pipe.fit(X_train, y_train)

y_test_pred = pipe.predict(X_test)
y_train_pred = pipe.predict(X_train)

print("r2 training: ", metrics.r2_score(y_train, y_train_pred))
print("r2 test: ", metrics.r2_score(y_test, y_test_pred))

print("rmse train: ", metrics.mean_squared_error(y_train, y_train_pred) ** 0.5)
print("rmse test: ", metrics.mean_squared_error(y_test, y_test_pred)** 0.5)

est = pipe.steps[-1][-1]
thetas = pd.DataFrame({"feature": X_dummy.columns
                       , "coeff":  est.coef_})
thetas["abs_coeff"] = np.abs(thetas["coeff"])
thetas.sort_values("abs_coeff", ascending=False)



r2 training:  0.8661986333575793
r2 test:  0.8788577151629232
rmse train:  0.14002970098567769
rmse test:  0.1513096787683941


,feature,coeff,abs_coeff
0,GrLivArea,0.131078,0.131078
1,OverallQual,0.111597,0.111597
3,YearBuilt,0.080856,0.080856
2,GarageCars,0.062827,0.062827
4,OverallCond,0.055581,0.055581
5,BsmtFullBath,0.041130,0.041130
21,Neighborhood_NridgHt,0.039886,0.039886
11,Neighborhood_Crawfor,0.032228,0.032228
27,Neighborhood_StoneBr,0.024543,0.024543
9,Neighborhood_ClearCr,0.024113,0.024113


In [52]:
cat_features = [c  for c in df.dtypes.index if str(df.dtypes[c]) == "object"]
num_features = [c  for c in df.dtypes.index if str(df.dtypes[c]) != "object"]
print("categorical columns: " , cat_features)
print("numeric columns: " , num_features)

categorical columns:  ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
numeric columns:  ['MSSubClass', 'LotFrontage', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt', 'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeck

In [ ]:
df = pd.read_csv("/data/kaggle/data_combined_cleaned.csv")
df = df.drop(columns="Id").dropna()
df.head()

In [ ]:
from sklearn import *

target = "SalesPrice"
X = df.drop(columns=target)
y = df[target]

cat_columns = [c  for c in X.dtypes.index if str(X.dtypes[c]) == "object"]
num_columns = [c  for c in X.dtypes.index if str(X.dtypes[c]) != "object"]

# Build pipelines for categorical data and numeric data
cat_pipe = pipeline.Pipeline([
    ('imputer', impute.SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', preprocessing.OneHotEncoder(handle_unknown='ignore'))
]) 

num_pipe = pipeline.Pipeline([
    ('imputer', impute.SimpleImputer(strategy='median')),
    ('poly', preprocessing.PolynomialFeatures(degree=1, include_bias=False)),
    ('scaler', preprocessing.StandardScaler()),
])

preprocessing_pipe = compose.ColumnTransformer([
    ("cat", cat_pipe, cat_columns),
    ("num", num_pipe, num_columns)
])


# Build estimator pipeline
estimator_pipe = pipeline.Pipeline([
    ("preprocessing", preprocessing_pipe),
    ("est", linear_model.ElasticNet(random_state=1))
])


# Parameter grid to tune hyper parameters
param_grid = {
    "est__alpha": 0.0 + np.random.random(10) * 0.02,
    "est__l1_ratio": np.linspace(0.0001, 1, 20),
}

# Grid Search estimator
gsearch = model_selection.GridSearchCV(estimator_pipe
                    , param_grid, cv = 5, verbose=1, n_jobs=8)



# Fit grid search estimator
print("Fitting the model")
gsearch.fit(X, y)

print("Gridsearch best score: ", gsearch.best_score_
      , "best params: ", gsearch.best_params_)

# Sanity test the quality of model
y_pred = gsearch.predict(X)
plt.scatter(y, y_pred - y)

print("Sample predictions: ", pd.DataFrame({"actual": y, "predict": y_pred}).sample(10))
